In [1]:
import requests
import pandas as pd
import time
import datetime as dt

In [26]:
# initial query to create file with df for subreddit 1: drake
s_type = 'comment' #or 'submission'
subreddit = 'drizzy'
unix_time_stamp = 1626888790

url = f'https://api.pushshift.io/reddit/search/{s_type}/?subreddit={subreddit}&before={unix_time_stamp}&size=100'
res = requests.get(url)
post_list = res.json()['data']
full_df = pd.DataFrame(post_list)[['body','author','created_utc','subreddit']]
full_df.to_csv('drizzy_comments.csv')

In [32]:
#function that pulls 100 rows of subreddit data * the amount of iterations specified for either comments or submissions
def pushshift_query(file,subreddit,s_type,iters=5):
    for pull in range(iters):
        full_df = pd.read_csv(file)
        url = f'https://api.pushshift.io/reddit/search/{s_type}/?subreddit={subreddit}&before={full_df["created_utc"].min()}&size=100'
        res = requests.get(url)
        assert res.status_code == 200
        post_list = res.json()['data']
        if s_type == 'submission':
            temp_df = pd.DataFrame(post_list)[['title','author','selftext','created_utc','subreddit']]
        elif s_type == 'comment':
            temp_df = pd.DataFrame(post_list)[['body','author','created_utc','subreddit']]
        full_df = pd.concat([full_df,temp_df])
        print(full_df.shape)
        print(full_df.nunique())
        print('waiting until next pull')
        time.sleep(5)
        full_df.to_csv(file,index=False)

#### Subreddit 1 (Drake) Data Pull

In [ ]:
#pulling data for drake subreddit
pushshift_query('drizzy_comments.csv','drizzy','comment',iters=20)

#### Subreddit 2 (Kanye) Data Pull

In [37]:
# initial query to create file with df for subreddit 2: kanye
s_type = 'comment' #or 'submission'
subreddit = 'kanye'
unix_time_stamp = 1626888790

url = f'https://api.pushshift.io/reddit/search/{s_type}/?subreddit={subreddit}&before={unix_time_stamp}&size=100'
res = requests.get(url)
post_list = res.json()['data']
full_df = pd.DataFrame(post_list)[['body','author','created_utc','subreddit']]
full_df.to_csv('kanye_comments.csv')

#### Cleaning and Fixing Data Collection Error

In [96]:
d = pd.read_csv('kanye_comments.csv')

In [97]:
d.shape

(2001, 5)

In [98]:
#checking to see error with created_utc values
d.sort_values('created_utc',ascending=False)

,Unnamed: 0,body,author,created_utc,subreddit
1972,: Silence,compotademansana,1626740116,Kanye,NaN
0,0.0,1 is pretty cool but i prefer 2 it’s just so b...,Jtemelsooo,1626805100,Kanye
1,1.0,Yeah you definitely shouldn't 🥴,Royal_Seaworthiness3,1626805096,Kanye
2,2.0,Very possible,Sky_Screamer18,1626805076,Kanye
3,3.0,[removed],[deleted],1626805051,Kanye
...,...,...,...,...,...
1997,NaN,[deleted],[deleted],1626739512,Kanye
1998,NaN,[removed],[deleted],1626739505,Kanye
1999,NaN,Neither does the new one imo. Still both great...,CRFcork,1626739479,Kanye
2000,NaN,Great art,Guccikilla1,1626739477,Kanye


In [99]:
#found incorrectly inputted data for this observation
d.iloc[1972]

Unnamed: 0            : Silence
body           compotademansana
author               1626740116
created_utc               Kanye
subreddit                   NaN
Name: 1972, dtype: object

In [100]:
#fixing data input error
import numpy as np
d.iloc[1972]['Unnamed: 0'] = np.nan
d.iloc[1972]['body'] = ': Silence'
d.iloc[1972]['author'] = 'compotademansana'
d.iloc[1972]['created_utc'] = '1626740116'
d.iloc[1972]['subreddit'] = 'Kanye'

In [101]:
#deleting bad observation
d.drop(labels=1971,inplace=True)

In [102]:
d.shape

(2000, 5)

In [103]:
#d.to_csv('kanye_comments.csv')

#### Query to pull r/kanye data

In [ ]:
#pulling data for kanye subreddit
pushshift_query('kanye_comments.csv','kanye','comment',iters=10)